In [5]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn_pandas import DataFrameMapper 
from scipy.integrate import trapz
import torch
import torchtuples as tt
from pycox.datasets import metabric
from pycox.models import CoxPH
from pycox.evaluation import EvalSurv
from preprocess import train_val_test_stratified_split
from survival_evaluation import d_calibration, l1, one_calibration
import random
import statistics

In [25]:
path = 'data_75063.csv'
d=pd.read_csv(path)
x_cols = ['age', 'sex']
event_col=['event']
time_col = ['time']
cols_standardize = ['age']
cols_leave = ['sex']
d['sex'] = d['sex'].replace(['female', 'male'], [0,1])

CI = []
IBS = []
L1_hinge = []
L1_margin = []

In [26]:
def train_val_test_stratified_split(df, stratify_colname='y', frac_train=0.6, frac_val=0.15, frac_test=0.25, random_state=None):
    if frac_train + frac_val + frac_test != 1.0:
        raise ValueError('fractions %f, %f, %f do not add up to 1.0' % (frac_train, frac_val, frac_test))
    if stratify_colname not in df.columns:
        raise ValueError('%s is not a column in the dataframe' % (stratify_colname))
    X = df # Contains all columns.
    y = df[[stratify_colname]] # Dataframe of just the column on which to stratify.
    df_train, df_temp, y_train, y_temp = train_test_split(X, y, stratify=y, test_size=(1.0 - frac_train), random_state=random_state)   
    relative_frac_test = frac_test / (frac_val + frac_test)
    if relative_frac_test == 1.0:
        df_val, df_test, y_val, y_test = [], df_temp, [], y_temp
    else:
        df_val, df_test, y_val, y_test = train_test_split(df_temp, y_temp, stratify=y_temp, test_size=relative_frac_test, random_state=random_state)
    assert len(df) == len(df_train) + len(df_val) + len(df_test)
    return df_train, df_val, df_test

In [27]:
def CoxPH_():
    df_train, df_val, df_test = train_val_test_stratified_split(d, 'event', frac_train=0.75, frac_val=0.05, frac_test=0.2)
    standardize = [([col], StandardScaler()) for col in cols_standardize]
    leave = [(col, None) for col in cols_leave]
    x_mapper = DataFrameMapper(standardize + leave)
    x_train = x_mapper.fit_transform(df_train).astype('float32')
    x_val = x_mapper.transform(df_val).astype('float32')
    x_test = x_mapper.transform(df_test).astype('float32')
    in_features = x_train.shape[1]
    
    num_durations = 10
    get_target = lambda df: (df['time'].values, df['event'].values)
    y_train = get_target(df_train)
    y_val = get_target(df_val)

    val = (x_val, y_val)
    durations_test, events_test = get_target(df_test)

    in_features = x_train.shape[1]
    out_features = 1
    num_nodes = [32, 32]
    batch_norm = True
    dropout = 0.1
    output_bias = False

    net = tt.practical.MLPVanilla(in_features, num_nodes, out_features, batch_norm,
                              dropout, output_bias=output_bias)
    
    model = CoxPH(net, tt.optim.Adam)

    batch_size = 256
    lr_finder = model.lr_finder(x_train, y_train, batch_size, tolerance=10)
    lr_finder.get_best_lr()
    model.optimizer.set_lr(0.01)

    epochs = 512
    callbacks = [tt.cb.EarlyStopping()]
    model.fit(x_train, y_train, batch_size, epochs, callbacks, val_data=val, verbose=0, val_batch_size=batch_size)

    _ = model.compute_baseline_hazards()
    surv = model.predict_surv_df(x_test)
    
    survival_predictions = pd.Series(trapz(surv.values.T, surv.index), index=df_test.index)
    l1_hinge = l1(df_test.time, df_test.event, survival_predictions, l1_type = 'hinge')
    l1_margin = l1(df_test.time, df_test.event, survival_predictions, df_train.time, df_train.event, l1_type = 'margin')

    ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')
    c_index = ev.concordance_td('antolini')
    time_grid = np.linspace(durations_test.min(), durations_test.max(), 100)
    brier = ev.integrated_brier_score(time_grid)

    return l1_hinge, l1_margin, c_index, brier, surv, df_test

In [28]:
def view_results(num_experiments):
    for i in range(num_experiments):
        l1_hinge, l1_margin, c_index, brier, surv, df_test = CoxPH_()
        L1_hinge.append(l1_hinge)
        L1_margin.append(l1_margin)
        CI.append(c_index)
        IBS.append(brier)
    print('L1_hinge: {:.3f} ({:.3f})\n'.format(statistics.mean(L1_hinge), statistics.stdev(L1_hinge)))
    print('L1_margin: {:.3f} ({:.3f})\n'.format(statistics.mean(L1_margin), statistics.stdev(L1_margin)))
    print('CI: {:.3f} ({:.3f})\n'.format(statistics.mean(CI), statistics.stdev(CI)))
    print('IBS: {:.3f} ({:.3f})\n'.format(statistics.mean(IBS), statistics.stdev(IBS)))
    
    # not yet considered the average value for d-calibration, df_test and surv are yielded at the last experiment loop.
    d_cal = d_calibration(df_test.event, surv.iloc[6])
    print('d-calibration: \n')
    print('p_value: {:.4f}\n'.format(d_cal['p_value']))
    print('bin_proportions:')
    for i in d_cal['bin_proportions']:
        print(i)
    print('\n')
    print('censored_contributions:')
    for i in d_cal['censored_contributions']:
        print(i)
    print('\n')
    print('uncensored_contributions:')
    for i in d_cal['uncensored_contributions']:
        print(i)

In [29]:
# how many experiment runs (should be >=2) to produce the final average results
view_results(2)

L1_hinge: 1.715 (0.056)

L1_margin: 142.096 (5.890)

CI: 0.811 (0.004)

IBS: 0.107 (0.012)

d-calibration: 

p_value: 0.0000

bin_proportions:
0.09805598646838715
0.09805598646838715
0.09805598646838715
0.09805598646838715
0.09805598646838715
0.09805598646838715
0.09805395328802455
0.09809636889192747
0.10018249321933062
0.11533126579039459


censored_contributions:
0.09805598646838715
0.09805598646838715
0.09805598646838715
0.09805598646838715
0.09805598646838715
0.09805598646838715
0.09805395328802455
0.09802975995300786
0.09778457141822491
0.08942038855066901


uncensored_contributions:
0.0
0.0
0.0
0.0
0.0
0.0
0.0
6.6608938919603e-05
0.0023979218011057086
0.02591087723972557
